In [112]:
import dask.array as da
import numpy as np
import xarray as xr
import rioxarray
from rioxarray.merge import merge_arrays

import boto3
import os
import time

In [81]:
complete_folder = 'landcover/composite/2020/raw/'

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('gfw2-data')

In [103]:
tile_ids = []

for object_summary in my_bucket.objects.filter(Prefix=complete_folder):
    tile_ids.append(object_summary.key[-12:-4])

tile_ids = tile_ids[1:]

print(tile_ids)

['00N_000E', '00N_010E', '00N_020E', '00N_030E', '00N_040E', '00N_040W', '00N_050W', '00N_060W', '00N_070W', '00N_080W', '00N_090E', '00N_090W', '00N_100E', '00N_100W', '00N_110E', '00N_120E', '00N_130E', '00N_140E', '00N_150E', '00N_160E', '10N_000E', '10N_010E', '10N_010W', '10N_020E', '10N_020W', '10N_030E', '10N_040E', '10N_050E', '10N_050W', '10N_060W', '10N_070E', '10N_070W', '10N_080E', '10N_080W', '10N_090E', '10N_090W', '10N_100E', '10N_100W', '10N_110E', '10N_120E', '10N_130E', '10S_010E', '10S_020E', '10S_030E', '10S_040E', '10S_040W', '10S_050E', '10S_050W', '10S_060W', '10S_070W', '10S_080W', '10S_110E', '10S_120E', '10S_130E', '10S_140E', '10S_150E', '10S_160E', '10S_170E', '20N_000E', '20N_010E', '20N_010W', '20N_020E', '20N_020W', '20N_030E', '20N_030W', '20N_040E', '20N_050E', '20N_060W', '20N_070E', '20N_070W', '20N_080E', '20N_080W', '20N_090E', '20N_090W', '20N_100E', '20N_100W', '20N_110E', '20N_110W', '20N_120E', '20N_160W', '20S_010E', '20S_020E', '20S_030E', '20

In [113]:
for year in list(range(2005, 2021, 5)):

    small_raster_paths = []
    
    change_folder = f"climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/{year-5}_{year}/2000_pixels/{time.strftime('%Y%m%d')}/"
   
    for object_summary in my_bucket.objects.filter(Prefix=change_folder):
        small_raster_paths.append(object_summary.key)
    
    # print(small_raster_paths)
    
    year_string = "IPCC_change_2000"
    tile_string = "50N_010E"
    
    # small_raster_paths = [i for i in small_raster_paths if year_string in i]
    small_raster_paths = [i for i in small_raster_paths if tile_string in i]
    
    # print(small_raster_paths)
    
    small_raster_paths = ['s3://gfw2-data/' + path for path in small_raster_paths]
    
    print(small_raster_paths)

['s3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2000_2005/2000_pixels/20240126/50N_010E__10_49_10_50__IPCC_change_2000_2005__20240126_14_50_30.tif', 's3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2000_2005/2000_pixels/20240126/50N_010E__10_49_11_50__IPCC_change_2000_2005__20240126_14_50_31.tif', 's3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2000_2005/2000_pixels/20240126/50N_010E__10_50_10_50__IPCC_change_2000_2005__20240126_14_50_35.tif', 's3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2000_2005/2000_pixels/20240126/50N_010E__10_50_11_50__IPCC_change_2000_2005__20240126_14_50_27.tif']
['s3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2005_2010/2000_pixels/20240126/50N_010E__10_49_10_50__IPCC_change_2005_2010__20240126_14_50_31.tif', 's3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2005_2010/2000_pixels/20240126/50N_010E__10_49_11

In [105]:
small_rasters = [rioxarray.open_rasterio(path, chunks=True) for path in small_raster_paths]
# small_rasters

In [106]:
merged = merge_arrays(small_rasters)  # https://corteva.github.io/rioxarray/stable/examples/merge.html
merged

<xarray.DataArray (band: 1, y: 4000, x: 4000)>
array([[[ 8,  8,  8, ...,  8,  8, 29],
        [ 8,  8,  8, ...,  8,  8,  8],
        [ 8,  8,  8, ...,  8,  8,  8],
        ...,
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ..., 15,  1,  1],
        [ 1,  1,  1, ..., 15,  1,  1]]], dtype=uint8)
Coordinates:
  * x            (x) float64 10.0 10.0 10.0 10.0 10.0 ... 11.0 11.0 11.0 11.0
  * y            (y) float64 50.0 50.0 50.0 50.0 50.0 ... 49.0 49.0 49.0 49.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [34]:
merged.rio.to_raster('/tmp/merged_change_2000.tif')

In [ ]:
s3_client.upload_file('/tmp/merged_change_2000.tif', "gfw2-data", Key=f"{s3_out_dir}/{value[2]}/{file_name}.tif")

# # Deletes the local raster. It won't be used again.
# os.remove(f"/tmp/{file_name}.tif")

In [17]:
# small_rasters = [xr.open_dataset(path, engine="rasterio") for path in small_raster_paths]

rioxarray.open_rasterio('./50N_010E__10_49_10_50__IPCC_change_2000_2005__20240126_13_08_25.tif', chunks=True)

<xarray.DataArray (band: 1, y: 2000, x: 2000)>
dask.array<open_rasterio-e8d2af56ac0dc367a059e1e58ae5ad97<this-array>, shape=(1, 2000, 2000), dtype=uint8, chunksize=(1, 2000, 2000), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 10.0 10.0 10.0 10.0 10.0 ... 10.5 10.5 10.5 10.5
  * y            (y) float64 49.5 49.5 49.5 49.5 49.5 ... 49.0 49.0 49.0 49.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [47]:
small_raster_paths = [f'{s3_out_dir}/IPCC_basic_change/50N_010E__10_49_10_50__IPCC_change_2000_2005__20240126_13_08_25.tif', 
                      f'{s3_out_dir}/IPCC_basic_change/50N_010E__10_49_11_50__IPCC_change_2000_2005__20240126_13_08_21.tif',
                      f'{s3_out_dir}/IPCC_basic_change/50N_010E__10_50_10_50__IPCC_change_2000_2005__20240126_13_08_24.tif',
                      f'{s3_out_dir}/IPCC_basic_change/50N_010E__10_50_11_50__IPCC_change_2000_2005__20240126_13_08_29.tif'
                    ]